# Nivel de paralelismo de Spark
Manipulación del nivel de paralelismo de Spark en máquina virtual: se usa la [definición de master](https://spark.apache.org/docs/latest/submitting-applications.html) como `local[k]`

In [ ]:
spark

In [ ]:
import time

def nothing(a):
    '''espera una milésima de segundo sin hacer nada'''
    time.sleep(0.001)
    return a+2

Fuerzo a 2 hebras. Tenemos que detener el contexto de Spark y crear otro con el nuevo master

In [ ]:
sc.stop()

sc = SparkContext(master='local[2]')

In [ ]:
a = sc.range(10000)
print("Partitions:", a.getNumPartitions())

a2 = a.repartition(4)

a3 = a2.map(lambda a: a+1)

a4 = a3.map(nothing)

%time a5 = a4.reduce( lambda a,b: a+b)

Ahora 4 hebras

In [ ]:
sc.stop()
sc = SparkContext(master='local[4]')

In [ ]:
a = sc.range(10000)
print("Partitions:", a.getNumPartitions())

a2 = a.repartition(4)

a3 = a2.map(lambda a: a+1)

a4 = a3.map(nothing)

%time a5 = a4.reduce( lambda a,b: a+b)

Y 8 hebras. La reducción de tiempo no es tan significativa ahora porque, aunque las operaciones de _map_ sí se realizan con 8 hebras, el _reduce_ se efectúa con 4 (probablemente por decisión del scheduler de Spark)

In [ ]:
sc.stop()
sc = SparkContext(master='local[8]')

In [ ]:
a = sc.range(10000)
print("Partitions:", a.getNumPartitions())

a2 = a.repartition(4)

a3 = a2.map(lambda a: a+1)

a4 = a3.map(nothing)

%time a5 = a4.reduce( lambda a,b: a+b)

El uso de `spark.cores.max`, en cambio, no funciona, puesto que es una [opción de configuración](https://spark.apache.org/docs/latest/configuration.html#scheduling) válida para Standalone o Mesos, pero no para un master local

In [ ]:
# Esto no funciona
conf2 = (SparkConf()
         .setMaster("local")
         .setAppName("PruebaSensores")
         .set("spark.cores.max", "2"))
sc.stop()
sc = SparkContext(conf=conf2)

In [ ]:
a = sc.range(10000)
print("Partitions:", a.getNumPartitions())

a2 = a.repartition(4)

a3 = a2.map(lambda a: a+1)

a4 = a3.map(nothing)

%time a5 = a4.reduce( lambda a,b: a+b)